<a href="https://colab.research.google.com/github/antonychackotc/machine-learning/blob/main/Activity_4(jan_10)_automatically_find_classifier_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **upload a file then train model on uploaded file data its Automatically find Linear Reggressor or decession tree Regressor or logistic Regressor or Decession tree classifier based on uploaded file character**

In [1]:
!pip install -q streamlit
!pip install -q pyngrok
!pip install -q localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [2]:
%%writefile app1.py

import streamlit as st
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Title and description
st.title("Dataset Upload and Auto Model Selection")
st.write("Upload your dataset, and the app will automatically select the appropriate model based on the target column's characteristics.")

# Step 1: Upload dataset
uploaded_file = st.file_uploader("Upload your dataset (CSV file)", type=["csv"])

if uploaded_file is not None:
    try:
        # Load the dataset into a Pandas DataFrame
        data = pd.read_csv(uploaded_file)
        st.write("### Dataset Preview:")
        st.dataframe(data.head())

        # Step 2: Select feature and target columns
        st.write("### Select your feature columns and target column")

        # Multiselect for feature columns
        feature_columns = st.multiselect("Choose feature columns (independent variables):", options=data.columns)

        # Dropdown for target column
        target_column = st.selectbox("Choose the target column (dependent variable):", options=data.columns)

        if feature_columns and target_column:
            st.write(f"You selected the following feature columns: {feature_columns}")
            st.write(f"You selected '{target_column}' as your target column.")

            # Determine the type of target column (categorical or numeric)
            if pd.api.types.is_numeric_dtype(data[target_column]):
                st.write("Target column is numeric. Selecting a regression model.")
                model_choice = "Regression"
            else:
                st.write("Target column is categorical. Selecting a classification model.")
                model_choice = "Classification"

            # Prepare features (X) and target (y)
            X = data[feature_columns]
            y = data[target_column]

            # Split the data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            if model_choice == "Regression":
                # Train both regression models
                linear_model = LinearRegression()
                decision_tree_regressor = DecisionTreeRegressor()

                linear_model.fit(X_train, y_train)
                decision_tree_regressor.fit(X_train, y_train)

                # Evaluate the models
                y_pred_linear = linear_model.predict(X_test)
                y_pred_tree = decision_tree_regressor.predict(X_test)

                mse_linear = mean_squared_error(y_test, y_pred_linear)
                mse_tree = mean_squared_error(y_test, y_pred_tree)

                st.write(f"### Linear Regression MSE: {mse_linear:.2f}")
                st.write(f"### Decision Tree Regressor MSE: {mse_tree:.2f}")

                # Select the best model
                if mse_linear < mse_tree:
                    selected_model = linear_model
                    st.write("Selected Model: Linear Regression")
                else:
                    selected_model = decision_tree_regressor
                    st.write("Selected Model: Decision Tree Regressor")

            elif model_choice == "Classification":
                # Encode target variable if categorical
                le = LabelEncoder()
                y = le.fit_transform(y)

                # Train both classification models
                logistic_model = LogisticRegression()
                decision_tree_classifier = DecisionTreeClassifier()

                logistic_model.fit(X_train, y_train)
                decision_tree_classifier.fit(X_train, y_train)

                # Evaluate the models
                y_pred_logistic = logistic_model.predict(X_test)
                y_pred_tree = decision_tree_classifier.predict(X_test)

                acc_logistic = accuracy_score(y_test, y_pred_logistic)
                acc_tree = accuracy_score(y_test, y_pred_tree)

                st.write(f"### Logistic Regression Accuracy: {acc_logistic:.2f}")
                st.write(f"### Decision Tree Classifier Accuracy: {acc_tree:.2f}")

                # Select the best model
                if acc_logistic > acc_tree:
                    selected_model = logistic_model
                    st.write("Selected Model: Logistic Regression")
                else:
                    selected_model = decision_tree_classifier
                    st.write("Selected Model: Decision Tree Classifier")

            # Prediction section
            st.write("### Enter data for prediction")
            input_data = {}
            for col in feature_columns:
                input_data[col] = st.number_input(f"Enter value for {col}", value=0.0)

            # Convert input data to DataFrame for prediction
            input_df = pd.DataFrame([input_data])

            if model_choice == "Regression":
                prediction = selected_model.predict(input_df)[0]
                st.write(f"### Prediction: {prediction:.2f}")
            elif model_choice == "Classification":
                prediction = le.inverse_transform([selected_model.predict(input_df)[0]])[0]
                st.write(f"### Prediction: {prediction}")

    except Exception as e:
        st.error(f"An error occurred: {e}")
else:
    st.write("Please upload a dataset to get started.")


Writing app1.py


In [ ]:
from pyngrok import ngrok

# Replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken
ngrok.set_auth_token("2rI2XurhgC2fxlYDtteHntWpCJf_5b1kDx2SLmwgq8GukDEyc")

# Run the Streamlit app in the background
!streamlit run app1.py &>/dev/null&

# Create a public URL using ngrok
try:
    public_url = ngrok.connect(8501)
    print(f"Streamlit app is running at {public_url}")
except Exception as e:
    print(f"Error: {e}")
    print("Trying to run with localtunnel")
    !streamlit run app1.py &>/content/logs.txt & npx localtunnel --port 8501